### WordPiece guideline

* Init vocab :
    * Characters at the beginning of each word
    * Characters in each word preceded by the prefix '##'
    * Special tokens
    * Compute pairs in each word with their respective frequency

* Objects : 
    * Vocab : list containing the vocab
    * Splits : dictionary containing pairs (updated) with their frequency in each word

* Iteration : 
    * Merge pairs with the highest score (arbitrary choice equality)
    * Until size_vocab reached

In [ ]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

### Code Review
**Defaultdict** : Python dictionnary which values can be initialized even if key not already present <br>
**set.update** : Insert multiple items at once in a set

In [10]:
from typing import List, Tuple
from collections import defaultdict as dd

def init_vocab(text: List[str]) -> Tuple[list, list, list]:
    special_tokens = ['[CLS]','[SEP]','[PAD]','[UNK]','[MASK]']
    vocab = set()
    word_freq = dd(int)
    splits = dd(list)

    for sentence in text:
        words = sentence.split(" ")
        for word in words:
            word_freq[word] += 1
            if word not in splits:
                splits[word] = [word[0]] + [f"##{c}" for c in word[1:]]
                vocab.update(splits[word])

    return special_tokens + list(vocab), word_freq, splits


vocab, word_freq, splits = init_vocab(corpus)

In [ ]:
from collections import defaultdict # for typing purposes

def compute_init_scores(splits: dict) -> defaultdict:
    # TODO : iterate through words to find pair_freq and individual_freq and compute scores for each pair